In [ ]:
import os
import sys
sys.path.append('/Users/david/Desktop/FinetuneEmbed')
import json
import pickle
import mygene
# Specify the working directory
os.chdir('/Users/david/Desktop/FinetuneEmbed')
# os.chdir('/afs/crc.nd.edu/group/StatDataMine/dm011/Dailin_Gan/FinetuneEmbed')
# load necessary functions
from mod.utils import *

# load the gene name embeddings
with open(f"./data/embeddings/GeneName_embed.pickle", "rb") as fp:
    gene_name_embeddings = pickle.load(fp)

random_states = list(range(41, 51)) # set up the random seeds
eva_size = 0.1 # validation data proportions
test_size = 0.1 # test data proportions


In [2]:
### Long- vs short- range TFs----------------------------------------------------------------
# The input data used here are downloaded from the geneformer paper Hugging Face website 
# (link: https://huggingface.co/datasets/ctheodoris/Genecorpus-30M/tree/main/example_input_files/gene_classification).
with open("./data/long_vs_shortTF/example_input_files_gene_classification_tf_regulatory_range_tf_regulatory_range.pickle", "rb") as f:
    check_data = pickle.load(f)

long_range_tf_gene = check_data['long_range']
short_range_tf_gene = check_data['short_range']

# convert gene id to gene symbols
mg = mygene.MyGeneInfo()
long_range_query = mg.querymany(long_range_tf_gene, species='human')
short_range_query = mg.querymany(short_range_tf_gene, species='human')
long_range_gene_name = [x['symbol'] for x in long_range_query]
short_range_gene_name = [x['symbol'] for x in short_range_query if 'symbol' in x]


2 input query terms found no hit:	['ENSG00000269603', 'ENSG00000267841']


In [7]:
long_range_tf_gene = list(set(long_range_gene_name) & set(gene_name_embeddings.keys())) # find the intersected genes
short_range_tf_gene = list(set(short_range_gene_name) & set(gene_name_embeddings.keys())) # find the intersected genes

In [12]:
genes = long_range_tf_gene + short_range_tf_gene
# 1 for long-range TF, 0 for short-range TF
labels = [1] * len(long_range_tf_gene) + [0] * len(short_range_tf_gene) 
save_dir = "./data/long_vs_shortTF/TrainEvalTestData_Name"
# Create the directory if it does not exist
os.makedirs(save_dir, exist_ok=True)

In [13]:
random_state = 41
TrainEvalTest_Name_split(genes, labels, save_dir, 
                        test_size=test_size, eval_size=eva_size, random_state=random_state)